### Redispersing a well plate into another 
This notebook describes one way of re-dispersing a plate full of samples so that we can alleviate particle sedimentation.

In [1]:
# We first load few Python programs from opentrons and a pandas package to load CSV files.
import opentrons.simulate
import json 
import pandas as pd 

# A protocol is an opentrons Python class that serves as 
# a central component for controlling the robot. 
# Here we are running this in simulation mode so that we will mimick 
# exisitence of a robot conencted to the computer you are running your code in
protocol = opentrons.simulate.get_protocol_api('2.18')

# Directory location for any custom labware we use.
# In this case, we will have custom labware defined for a 6-well reservoir
CUSTOM_LABWARE_PATH = "./Custom_Labware/" 
protocol.home()

/Users/pozzolabadmin/.opentrons/robot_settings.json not found. Loading defaults
Deck calibration not found.
/Users/pozzolabadmin/.opentrons/deck_calibration.json not found. Loading defaults


In [10]:
# load all the labware modules
with open(CUSTOM_LABWARE_PATH+'agilent_6_reservoir_47000ul.json') as labware_file:
    reservoir_def = json.load(labware_file)
    reservoir = protocol.load_labware_from_definition(reservoir_def, location=1)

synthesized_plate = protocol.load_labware(
    load_name="corning_96_wellplate_360ul_flat",
    location=2)

tiprack_300 = protocol.load_labware(
    load_name="opentrons_96_tiprack_300ul",
    location=4)

p300 = protocol.load_instrument(
    instrument_name="p300_single_gen2",
    mount="right",
    tip_racks=[tiprack_300]
    )

In [11]:
# load volume array
volumes = pd.read_csv("./volumes_repeat_071724.csv")

In [12]:
nonempty_wells = []
counter = 1
for row in synthesized_plate.rows():
    for well in row:
        if counter>volumes.shape[0]:
            break
        nonempty_wells.append(well)
        counter += 1

In [17]:
# define the protocol
def redisperse(well, pipette):
    """ Redisperse liquid in a single well

    well : destination well Labware.well
    pipette : pipette module to use InstrumentContext
    """
    print("redispersing %s..."%well.well_name,end='\r', flush=True)
    pipette.mix(2, 30, well.bottom())
    pipette.blow_out()

    # clean the tip in a reservoir
    pipette.mix(2, 30, reservoir["A3"],rate=2)
    pipette.mix(2, 30, reservoir["A4"],rate=2)
    pipette.blow_out(reservoir["A5"])

    return
    
p300.pick_up_tip(tiprack_300["A1"])
for well in nonempty_wells:
    redisperse(well, p300)
p300.drop_tip(tiprack_300["A1"])

<InstrumentContext: p300_single_v2.1 in RIGHT>

In [15]:
# Output steps in protocol
for line in protocol.commands():
    print(line)

Picking up tip from A1 of Opentrons OT-2 96 Tip Rack 300 µL on slot 4
Mixing 2 times with a volume of 30.0 ul
Aspirating 30.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on slot 2 at 92.86 uL/sec
Dispensing 30.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on slot 2 at 92.86 uL/sec
Aspirating 30.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on slot 2 at 92.86 uL/sec
Dispensing 30.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on slot 2 at 92.86 uL/sec
Blowing out at A1 of Corning 96 Well Plate 360 µL Flat on slot 2
Mixing 2 times with a volume of 30.0 ul
Aspirating 30.0 uL from A3 of Agilent 6 Reservoir 47000 µL on slot 1 at 185.72 uL/sec
Dispensing 30.0 uL into A3 of Agilent 6 Reservoir 47000 µL on slot 1 at 185.72 uL/sec
Aspirating 30.0 uL from A3 of Agilent 6 Reservoir 47000 µL on slot 1 at 185.72 uL/sec
Dispensing 30.0 uL into A3 of Agilent 6 Reservoir 47000 µL on slot 1 at 185.72 uL/sec
Mixing 2 times with a volume of 30.0 ul
Aspirating 30.0 uL from A4 of Agilent 6 R